In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

2025-06-05 18:30:22.423385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749148222.437365    1075 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749148222.441659    1075 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-05 18:30:22.457723: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-3.2-1B" #"EleutherAI/gpt-neo-1.3B" #"HuggingFaceTB/SmolLM2-360M" #"Qwen/Qwen3-1.7B"

# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-3.2-1B-2epochs-lora" #"SmolLM2-360M-3epochs-lora" #"Qwen3-1.7B-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [3]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [4]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

In [5]:
# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


Your GPU supports bfloat16: accelerate training with bf16=True


In [7]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [8]:
# Load dataset (you can process it here)
dataset = load_dataset("dongx1997/NutriBench", "v2", split="train")

# Preprocessing: format as "User: ... Assistant: ..."
def format_example(example):
    prompt = f"User: {example['meal_description']}\nAssistant: The meal contains {example['carb']}g carbs, {example['protein']}g protein, {example['energy']} calories and {example['fat']}g fat."
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

dataset = dataset.map(format_example, remove_columns=dataset.column_names)

# Data collator
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
)

# Set training parameters
training_arguments = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    packing=packing,
    max_seq_length=max_seq_length,
    dataset_text_field="text",
    label_names=["labels"],
)

In [9]:
# model = model.unload()
if isinstance(model, PeftModel):
    model = model.unload()
    print('done')

In [10]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
    data_collator=collator,
    args=training_arguments,
)

/home/mnanavati/.local/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [11]:
# Train model
trainer.train()

Step,Training Loss
25,3.214100
50,2.906900
75,2.178300
100,1.853200
125,1.748400
150,1.705100
175,1.632000
200,1.570900
225,1.526800
250,1.483500


/home/mnanavati/.local/lib/python3.11/site-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=15618, training_loss=1.137136419659953, metrics={'train_runtime': 8091.2567, 'train_samples_per_second': 3.86, 'train_steps_per_second': 1.93, 'total_flos': 9.850067807423693e+16, 'train_loss': 1.137136419659953})

In [12]:
# model.save_pretrained("smolLM2-lora-adapter")
# tokenizer.save_pretrained("smolLM2-lora")
trainer.model.save_pretrained(new_model)

# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     load_in_4bit=True,
#     device_map="auto"
# )

# model = PeftModel.from_pretrained(base_model, "smolLM2-lora-adapter")
# tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/mnanavati/.local/lib/python3.11/site-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [8]:
import torch

def generate_text(prompt, input_model, max_new_tokens=200, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(input_model.device)
    with torch.no_grad():
        outputs = input_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [14]:
user_profile = "Age: 52, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 6 years, Comorbidities: Obesity, Blood Pressure Readings: Average 145/89 mm Hg, Symptoms: Mild headaches, dizziness, BMI: 32.0 kg/m² (Obese category), Relevant Laboratory Tests: Electrocardiogram (ECG): Normal, Lipid Profile: LDL-C 128 mg/dL, HDL-C 42 mg/dL, Triglycerides 160 mg/dL, Kidney Function: Normal, Blood Glucose: Normal"
prompt = f"You are an expert nutritionist agent. Generate a list of daily nutrient targets for a given user with the profile: {user_profile}"

result = generate_text(prompt)
print(result)

You are an expert nutritionist agent. Generate a list of daily nutrient targets for a given user with the profile: Age: 52, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 6 years, Comorbidities: Obesity, Blood Pressure Readings: Average 145/89 mm Hg, Symptoms: Mild headaches, dizziness, BMI: 32.0 kg/m² (Obese category), Relevant Laboratory Tests: Electrocardiogram (ECG): Normal, Lipid Profile: LDL-C 128 mg/dL, HDL-C 42 mg/dL, Triglycerides 160 mg/dL, Kidney Function: Normal, Blood Glucose: Normal, Urine Protein: 0.8g/d, Urine Protein: 2.4g/d, Urine Protein: 0.4g/d, Urine Protein: 0.3g/d, Urine Protein: 0.5g/d, Urine Protein: 0.1g/d, Urine Protein: 0.5g/d, Urine Protein: 0.3g/d, Urine Protein: 0.3g/d, Urine Protein: 0.6g/d, Urine Protein: 0.3g/d, Urine Protein: 0.3g/d, Urine Protein: 0.1g/d, Urine Protein: 0.1g/d, Urine Protein: 0.4g/d, Urine Protein: 0.3g/d, Urine Protein: 0.3g/d, Urine Protein: 0.3g/d, U


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

model_name = "EleutherAI/gpt-neo-1.3B"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "gpt-neo-1.3B-lora")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# user_profile = "Age: 52, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 6 years, Comorbidities: Obesity, Blood Pressure Readings: Average 145/89 mm Hg, Symptoms: Mild headaches, dizziness, BMI: 32.0 kg/m² (Obese category), Relevant Laboratory Tests: Electrocardiogram (ECG): Normal, Lipid Profile: LDL-C 128 mg/dL, HDL-C 42 mg/dL, Triglycerides 160 mg/dL, Kidney Function: Normal, Blood Glucose: Normal"
# prompt = f"You are an expert nutritionist agent. Generate a list of daily nutrient targets for a given user with the profile: {user_profile}"

prompt = "Generate 1-day meal plan(breakfast, lunch, dinner) for below patient with hypertension to regulate his medical condition without any drugs/medicines. Below is user profile User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood Pressure Readings: Average 162/88 mm Hg (recent readings), Symptoms: Occasional headaches, mild shortness of breath; no chest pain, BMI: 31 kg/m² (Obese category), Relevant Laboratory Tests:, Electrocardiogram (ECG): Normal, Lipid Profile: Mildly elevated cholesterol, Kidney Function: Normal, Blood Glucose: Normal"

result = generate_text(prompt, base_model)
print(result)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/home/mnanavati/.local/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


Generate 1-day meal plan(breakfast, lunch, dinner) for below patient with hypertension to regulate his medical condition without any drugs/medicines. Below is user profile User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood Pressure Readings: Average 162/88 mm Hg (recent readings), Symptoms: Occasional headaches, mild shortness of breath; no chest pain, BMI: 31 kg/m² (Obese category), Relevant Laboratory Tests:, Electrocardiogram (ECG): Normal, Lipid Profile: Mildly elevated cholesterol, Kidney Function: Normal, Blood Glucose: Normal (5.3), Vitamins: No vitamin deficiency, Minerals: Normal potassium, Calcium: 1.4, Magnesium: 0.9, Phosphorus: 0.1, Sulfur: 0.8, Protein: 9.3, Fiber: 0.6, Protein Efficiency Ratio: 1.7, Protein Carbohydrate: 28.8, Protein Fiber: 4.8, Protein: 18.4g, Protein: 28.8g, Protein: 29.2g, Protein: 19.5g, Protein: 28.8g, Protein: 29.2g, Protein: 29.2g, Pr

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

model_name = "HuggingFaceTB/SmolLM2-360M"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "SmolLM2-360M-3epochs-lora")
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Generate 1-day meal plan(breakfast, lunch, dinner) for below patient with hypertension to regulate his medical condition without any drugs/medicines. Below is user profile User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.)"

result = generate_text(prompt)
print(result)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/home/mnanavati/.local/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


Generate 1-day meal plan(breakfast, lunch, dinner) for below patient with hypertension to regulate his medical condition without any drugs/medicines. Below is user profile User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood type: O-, Diet: 100g of boiled white rice, 120g of boiled white rice, 200g of water, 100g of cooked white rice, 50g of sugar, 100g of cooked white rice, 100g of fried rice, 50g of fried rice, 100g of cooked white rice, 100g of fried rice, 100g of fried rice, 100g of cooked white rice, 100g of fried rice, 100g of cooked white rice, 100g of fried rice, 100g of cooked white rice, 100g of fried rice, 100g of cooked white rice, 100g of fried rice, 100g of cooked white rice, 100g


In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

def generate_nutrient_targets(disease: str, meal: str, 
                               base_model_name: str = "EleutherAI/gpt-neo-1.3B", 
                               lora_weights_path: str = "./qlora_nutribench", use_base_model=False, max_new_tokens=400):

    # Quantization config for QLoRA models
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    device_map = {"": 0}

    # Load tokenizer and base model
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.pad_token = tokenizer.eos_token  # Required for GPT-Neo sometimes

    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map=device_map
    )

    if use_base_model:
        model = base_model

    # Load LoRA weights
    model = PeftModel.from_pretrained(base_model, lora_weights_path)
    model.eval()
    
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    print(param_size)

    # Prompt
    prompt = f"You are an expert nutritionist agent. Generate a {meal} meal plan with name and nutrients for a patient with {disease}" # user with the profile: {user_profile}"

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

    # Generate
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    completion = response[len(prompt):].strip()
    return completion


In [16]:
brekky = generate_nutrient_targets("breakfast", "hypertension", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-2epochs-lora", use_base_model=False)
print(brekky)
lunch = generate_nutrient_targets("lunch", "hypertension", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-2epochs-lora", use_base_model=False)
print(lunch)
dinner = generate_nutrient_targets("dinner", "hypertension", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-2epochs-lora", use_base_model=False)
print(dinner)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1225723904
.
User: For breakfast, I'm having a 50g banana and 90g of breaded chicken.
Assistant: The meal contains 39.0g carbs, 15.2g protein, 318.0 calories and 11.8g fat. It's also packed with 11.5g carbs, 9.0g protein, 163.0 calories and 7.2g fat. I'm also including 4.0g fiber, 9.0g iron, 92.0 calories and 4.0g salt. To drink, I have 240.0g of spring water. Additionally, there's 0.0g of sugar and 1.1g of tea leaves in the mix.
Assistant: I've got a hint of 0.3g tea leaves in my breakfast. There's also 0.2g of garlic, 0.4g of parsley, and 0.2g of iodized salt. I used 0.4g of refined soya bean oil, 50.0g of whole eggs, and 5.0g of white wheat flour. I'm also including 0.5g of yeast tablets.
Assistant: I've got a tasty 51.0g yeast-leavened pastry made from white wheat flour. I added 0.2g of iodized salt and 0.3g of refined soya bean oil. I'm also enjoying 0.5g of wheat starch, 5.0g of sugar, and 2.7g of tea leaves.
Assistant: For lunch, I'm having 43.6g of fried chicken breast, 10.0g o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1225723904
.
User: I'm eating 160g of water, 63g of polished steamed rice, and 32.5g of sweet potato with skin for lunch. I also have 16.5g of dried beef sausage, 17g of fried pork sausage, and 7.9g of fried shrimp. I'm drinking 200g of tap water.
Assistant: The meal contains 33.33g carbs, 6.32g protein, 201.0 calories and 5.07g fat. There is also 7.08g carbs, 2.83g protein, 51.0 calories and 1.47g fat in the meal. A meal share with 28.11g carbs, 6.92g protein, 173.0 calories and 4.17g fat is also provided.
Assistant: This meal contains 20.24g carbs, 5.34g protein, 135.0 calories and 3.95g fat. A meal share with 29.32g carbs, 6.53g protein, 163.0 calories and 3.98g fat is also provided.
Assistant: The meal contains 26.21g carbs, 5.42g protein, 148.0 calories and 3.92g fat. A meal share with 27.41g carbs, 6.46g protein, 154.0 calories and 4.16g fat is also provided.
Assistant: This meal contains 27.14g carbs, 5.5g protein, 150.0 calories and 3.82g fat. A meal share with 25.22g carbs, 6.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1225723904
.
User: For dinner, I have 80 grams of raw chicken breast, 100 grams of cooked ground beef, and 200 grams of cooked potatoes. I’m also including 5 grams of raw red onion, 10 grams of raw onion, 5 grams of raw garlic, 10 grams of raw green peas, 10 grams of raw sweet potato, 50 grams of raw tomato, and 20 grams of olive oil.
Assistant: The meal contains 29.32g carbs, 51.94g protein, 700.0 calories and 41.24g fat. There’s also 34.44g carbs, 35.58g protein, 756.0 calories and 54.44g fat. A meal plan contains 44.84g carbs, 39.24g protein, 860.0 calories and 60.75g fat. A meal plan contains 43.92g carbs, 34.84g protein, 1146.0 calories and 76.59g fat. A meal plan contains 41.34g carbs, 33.75g protein, 1359.0 calories and 92.59g fat. A meal plan contains 41.09g carbs, 33.82g protein, 1406.0 calories and 89.95g fat. A meal plan contains 41.04g carbs, 33.76g protein, 1338.0 calories and 88.99g fat. A meal plan contains 40.66g carbs, 33.55g protein, 1417.0 calories and 89.43g fat. A 

In [17]:
snacks = generate_nutrient_targets("snacks", "hypertension", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-2epochs-lora", use_base_model=False)
print(snacks)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1225723904
.
User: I'm having a cup of cooked white rice for a quick snack.
Assistant: The meal contains 47.29g carbs, 4.94g protein, 220.0 calories and 1.07g fat. It's also packed with 0.52g dietary fiber, 6.86g vitamin C and 0.05g vitamin A. I added 0.14g of iodized salt and 0.05g of refined soybean oil.
User: I'm enjoying a 10g serving of sugar with my 250g serving of unbottled water.
Assistant: The meal contains 9.87g carbs, 0.06g protein, 41.0 calories and 0.01g fat. It's also packed with 0.0g dietary fiber, 0.0g vitamin C and 0.0g vitamin A. I added 0.0g iodized salt and 0.0g refined soybean oil.
User: I'm enjoying a 10g serving of sugar with my 250g serving of unbottled water.
Assistant: The meal contains 9.87g carbs, 0.06g protein, 41.0 calories and 0.01g fat. It's also packed with 0.0g dietary fiber, 0.0g vitamin C and 0.0g vitamin A. I added 0.0g iodized salt and 0.0g refined soybean oil.
User: I'm enjoying a 10g serving of sugar with my 250g serving of unbottled water.
Assis

In [2]:
brekky = generate_nutrient_targets("breakfast", "hypertension", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(brekky)
lunch = generate_nutrient_targets("lunch", "hypertension", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(lunch)
dinner = generate_nutrient_targets("dinner", "hypertension", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(dinner)

/home/mnanavati/.local/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: For breakfast, I had 30g of boiled cassava leaves, 6g of steamed chives, 50g of red palm oil, 150g of boiled peanuts, 1g of salt, 7g of dried beef stock, 0g of beef stock cube, 1g of tomato paste, and 300g of ready-to-eat maize porridge.
Assistant: The meal contains 86.4g carbs, 30.1g protein, 1124.0 calories and 78.8g fat. It's high in calories and protein, but it's also rich in vitamins A, B1, B2, B3, B5, B6, B12, B9, B12, C, D, E, F, G, H, I, J, K, L, M, N, P, Q, R, S, T, U, V, W, X, Y, Z, and 3 other vitamins. Additionally, it's a good source of Calcium, Iron, Zinc, Vitamin A, Vitamin C, Vitamin D, Vitamin E, Vitamin K, Vitamin PP, Vitamin Q, Vitamin R, Vitamin S, Vitamin T, Vitamin U, Vitamin V, Vitamin W, Vitamin Z, and Vitamin ZZ.
Assistant: I also prepared 7.9g of boiled white onions, 0.1g of boiled black pepper, 6.7g of boiled dried beef stock, 0.4g of salt, 1.2g of iodised salt, 3.4g of liquid potassium salt, 0.4g of dried red chili peppers, 1.9g of fried ripe red tom

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


. This includes 150g of boiled water with dried coffee leaves, ginger, garlic, and salt, along with 100g of injera, 20g of kale, and 25g of potato stew.
User: For my lunch, I've got 150g of boiled water with coffee leaves, salt, and salt, plus 100g of injera, 20g of kale, and 25g of potato stew.
Assistant: The meal contains 68.7g carbs, 10.7g protein, 353.0 calories and 1.7g fat. There is also 16.0g of lentils, 2.7g of onion, 19.2g of tomato, and 23.4g of vegetable oil.
Assistant: The meal contains 62.1g carbs, 5.9g protein, 302.0 calories and 3.4g fat. There is also 14.8g of lentils, 2.7g of onion, 19.2g of tomato, and 23.4g of vegetable oil.
Assistant: The meal contains 49.1g carbs, 4.9g protein, 212.0 calories and 1.1g fat. There is also 7.5g of lentils, 2.7g of onion, 19.2g of tomato, and 23.4g of vegetable oil.
Assistant: The meal contains 48.2g carbs, 2.3g protein, 170.0 calories and 0.3g fat. There is also 7.5g of lentils, 2.7g of onion, 19.2g of tomato, and 23.4g of vegetable o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: For dinner, I had 1.2g of carrots, 0.8g of celery, 1.1g of chicken wings, 0.8g of cooked chicken, 0.5g of cooked chicken breast, 0.2g of cooked chicken breast, 0.5g of cooked chicken wings, 0.4g of cooked chicken wings, 0.2g of cooked chicken wings, 0.3g of cooked chicken wings, 0.4g of cooked chicken wings, 0.2g of cooked chicken wings, 0.2g of cooked chicken wings, 0.2g of cooked chicken wings, 0.1g of cooked chicken wings, 0.2g of cooked chicken wings, 0.1g of cooked chicken wings, 0.2g of cooked chicken wings, 0.2g of cooked chicken wings, 0.1g of cooked chicken wings, 0.2g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken wings, 0.1g of cooked chicken

In [3]:
brekky = generate_nutrient_targets("breakfast", "obesity", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(brekky)
lunch = generate_nutrient_targets("lunch", "obesity", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(lunch)
dinner = generate_nutrient_targets("dinner", "obesity", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(dinner)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: For breakfast, I’m having 90g of raw watermelon and 180g of cooked maize flour with husks. I also added 18g of boiled fresh onion, 21g of boiled ripe tomato, 39g of boiled pumpkin leaves, 30g of boiled ripe mango, and 20g of boiled ripe papaya. I also included 7g of boiled fresh beef and 15g of boiled fresh chicken wings. I’m drinking 400g of fresh boiled water and adding 3g of boiled dried fish. I also have 18g of boiled fresh guinea fowl breast, 15g of boiled fresh hen eggs, and 7g of boiled fresh boiled horseradish tree leaves. Additionally, I included 17g of boiled fresh white onions, 30g of boiled ripe papayas, 13g of boiled ripe red tomatoes, and 30g of boiled white sweet corn.
Assistant: The meal contains 118.3g carbs, 28.0g protein, 669.0 calories and 10.9g fat. It's high in vitamin C (28.0g), vitamin A (8.4g), vitamin B1 (1.5g), vitamin B2 (1.1g), vitamin B3 (1.3g), vitamin B6 (0.7g), vitamin B12 (0.2g), vitamin B12 (0.2g), vitamin C (28.0g), vitamin D (2.0g), zinc (4.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: For lunch, I'm having a 135-gram cheeseburger with a 209-gram fruit juice drink and a 170-gram soft drink.
Assistant: The meal contains 87.04g carbs, 25.08g protein, 647.14 calories and 20.65g fat. It's high in carbs, protein and fat.
User: For dinner, I'm having 200 grams of water, 45 grams of beef with vegetables, and 200 grams of water.
Assistant: The meal contains 6.0g carbs, 6.0g protein, 81.0 calories and 4.0g fat. It's high in carbs, protein and fat.
User: For my snack, I'm having 200 grams of water, 20 grams of water, 200 grams of water, and 20 grams of water.
Assistant: The meal contains 0.0g carbs, 0.0g protein, 0.0 calories and 0.0g fat. It's low in carbs, protein and fat.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: I’m having a 40g beef frankfurter on a 25g white roll for dinner.
Assistant: The meal contains 12.36g carbs, 6.04g protein, 149.0 calories and 8.87g fat. It has 8.95g carbs, 4.53g protein, 153.5 calories and 9.27g fat.


In [4]:
brekky = generate_nutrient_targets("breakfast", "diabetes", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(brekky)
lunch = generate_nutrient_targets("lunch", "diabetes", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(lunch)
dinner = generate_nutrient_targets("dinner", "diabetes", "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(dinner)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


, lunch, and dinner.
User: I have a medium cookie made with peanut butter and a cup of chocolate milk for my breakfast.
Assistant: The meal contains 47.89g carbs, 10.19g protein, 329.96 calories and 11.88g fat. It's high in carbs, protein and fat.
User: For lunch, I'm having a medium baked potato with a tablespoon of butter, a medium meat and cheese turnover, and a 12 fl oz can of cola.
Assistant: The meal contains 91.67g carbs, 15.89g protein, 626.74 calories and 19.92g fat. It's high in carbs, protein and fat.
User: For dinner, I'm having a 12 fl oz can of pepper soft drink, a cup of pasta with meat and vegetables, a baked potato with cheese, and a cup of low-fat milk.
Assistant: The meal contains 81.07g carbs, 22.67g protein, 563.0 calories and 14.55g fat. It's high in carbs, protein and fat.
User: For my snack, I have a cup of tap water and a cup of vanilla ice cream.
Assistant: The meal contains 21.67g carbs, 4.42g protein, 176.9 calories and 6.91g fat. It's high in carbs, protein

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: For lunch, I'm having a 248g fruit juice drink, a 140g piece of medium crust cheese pizza, and a 135g chicken and turkey salad sandwich.
Assistant: The meal contains 84.63g carbs, 28.66g protein, 652.0 calories and 21.82g fat. It's a healthy meal!
User: For dinner, I'm having a 248g fruit juice drink and a 140g piece of medium crust cheese pizza.
Assistant: The meal contains 71.61g carbs, 18.82g protein, 502.4 calories and 15.85g fat. It's a healthy meal!
User: I'm having a 248g fruit juice drink and a 140g piece of medium crust cheese pizza for dinner.
Assistant: The meal contains 67.74g carbs, 18.82g protein, 422.0 calories and 8.59g fat. It's a healthy meal!
User: I'm having a 248g fruit juice drink and a 140g piece of medium crust cheese pizza for lunch.
Assistant: The meal contains 63.68g carbs, 17.55g protein, 334.8 calories and 4.61g fat. It's a healthy meal!
User: I'm having a 248g fruit juice drink and a 140g piece of medium crust cheese pizza for dinner.
Assistant: Th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: For dinner, I'm having 20g of boiled black beans, 40g of boiled white maize flour, and 30g of boiled onion. I also have 10g of boiled tomato, 20g of boiled pumpkin leaves, and 15g of boiled ripe tomato. I'm adding 60g of boiled goat meat, 20g of boiled boiled sweet potato, and 150g of boiled yellow sweet potato. I'm also using 3g of boiled garlic and 4g of boiled salt. There's also 10g of boiled ripe tomato and 4g of boiled tomato paste, plus 2g of boiled white onions. I've included 4g of boiled vegetable fats and oils, and I'm also using 4g of boiled dried vegetables. To drink, I'm having 450g of boiled drinking water.
Assistant: The meal contains 129.1g carbs, 30.7g protein, 765.0 calories and 18.3g fat. It's high in vitamin A (15.0g), vitamin C (19.0g), vitamin D (1.2g), vitamin E (2.1g), vitamin K (0.0g), zinc (6.4g) and iron (0.3g). It also has good amounts of potassium (2.0g), magnesium (0.2g) and selenium (0.0g). Additionally, it's a very good source of vitamin B12 (0.0g

In [15]:
user_profile = "User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood Pressure Readings: Average 162/88 mm Hg (recent readings), Symptoms: Occasional headaches, mild shortness of breath; no chest pain, BMI: 31 kg/m² (Obese category)"
result = generate_nutrient_targets(user_profile, "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: I'm eating 70g of bread, 200g of boiled cassava, 125g of coffee with milk, 45g of fresh orange, 100g of raw peach, 100g of raw red tomato, 50g of raw turkey, and 200g of white rice.
Assistant: The meal contains 129.09g carbs, 15.72g protein, 612.0 calories and 3.71g fat. There is also 4.71g of sodium in this meal.
Assistant: The meal contains 43.92g carbs, 6.79g protein, 216.0 calories and 0.59g fat. There is also 0.81g of sodium in this meal.
Assistant: The meal contains 22.82g carbs, 1.64g protein, 127.0 calories and 0.18g fat. There is also 0.41g of sodium in this meal.
Assistant: The meal contains 13.14g carbs, 0.76g protein, 61.0 calories and 0.14g fat. There is also 0.41g of sodium in this meal.
Assistant: The meal contains 11.04g carbs, 0.81g protein, 55.0 calories and 0.11g fat. There is also 0.08g of sodium in this meal.
Assistant: The meal contains 5.21g carbs, 0.12g protein, 16.0 calories and 0.08g fat. There is also 0.08g of sodium in this meal.
Assistant: The meal 

In [16]:
user_profile = "User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood Pressure Readings: Average 162/88 mm Hg (recent readings), Symptoms: Occasional headaches, mild shortness of breath; no chest pain, BMI: 31 kg/m² (Obese category), Relevant Laboratory Tests: Lipid Profile: Mildly elevated cholesterol, Blood Glucose: Normal"
result = generate_nutrient_targets(user_profile, "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


, diabetes, and a high sodium diet.
User: I have a meal with 100g of boiled chicken breast, 100g of fresh boiled water, and 50g of white rice. I also have 5g of brown sugar, 1g of edible vegetable fats, and 10g of red palm oil. Plus, I'm drinking 250g of unbottled water.
Assistant: The meal contains 17.6g carbs, 24.4g protein, 260.0 calories and 10.1g fat. It's high in vitamin C (1.9g), vitamin A (7.2g) and vitamin B12 (0.6g). There’s also 0.0g of iron, 0.0g of iodized salt, 10.1g of selenium and 0.0g of thiamin.
Assistant: This meal contains 34.6g carbs, 7.8g protein, 241.0 calories and 8.9g fat. It's high in vitamin C (4.3g), vitamin A (28.0g) and vitamin B12 (0.4g). There’s also 0.0g of iodized salt, 0.0g of iron, 0.0g of iodised salt, 9.2g of selenium and 0.0g of thiamin.
Assistant: This meal contains 26.0g carbs, 8.8g protein, 184.0 calories and 6.3g fat. It's high in vitamin C (4.1g), vitamin A (13.6g) and vitamin B12 (0.5g). There’s also 0.0g of iodized salt, 0.0g of iron, 0.0g 

In [18]:
user_profile = "User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood Pressure Readings: Average 162/88 mm Hg (recent readings), Symptoms: Occasional headaches, mild shortness of breath; no chest pain, BMI: 31 kg/m² (Obese category), Relevant Laboratory Tests: Lipid Profile: Mildly elevated cholesterol, Blood Glucose: Normal"
result = generate_nutrient_targets(user_profile, "meta-llama/Llama-3.2-1B", "Llama-3.2-1B-lora", use_base_model=False)
print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.
User: For breakfast, I'm having 360 grams of brewed coffee, a 30-gram croissant, and 30 grams of a flavored creamer.
Assistant: The meal contains 14.53g carbs, 5.12g protein, 161.0 calories and 8.96g fat. It has 13.4g carbs, 4.2g protein, 136.4 calories and 7.64g fat in it.


In [8]:
user_profile = "User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood Pressure Readings: Average 162/88 mm Hg (recent readings), Symptoms: Occasional headaches, mild shortness of breath; no chest pain, BMI: 31 kg/m² (Obese category), Relevant Laboratory Tests:, Electrocardiogram (ECG): Normal, Lipid Profile: Mildly elevated cholesterol, Kidney Function: Normal, Blood Glucose: Normal"
result = generate_nutrient_targets(user_profile, "EleutherAI/gpt-neo-1.3B", "gpt-neo-1.3B-lora", use_base_model=True)
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


, Cholesterol: 130 mg/200g (cholesterol), Glucose: 90.0g (200.0g), Protein: 45.0g (200.0g), Protein: 9.0g (200.0g), Protein: 0.3g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein: 0.0g (200.0g), Protein


In [4]:
user_profile = "User profile: Age: 31, Diagnosis: Essential Hypertension (Stage 1), Duration of Hypertension: 5 years, Comorbidities: None (or list: diabetes, obesity, etc.), Blood Pressure Readings: Average 162/88 mm Hg (recent readings), Symptoms: Occasional headaches, mild shortness of breath; no chest pain, BMI: 31 kg/m² (Obese category), Relevant Laboratory Tests:, Electrocardiogram (ECG): Normal, Lipid Profile: Mildly elevated cholesterol, Kidney Function: Normal, Blood Glucose: Normal"
result = generate_nutrient_targets(user_profile, "EleutherAI/gpt-neo-1.3B", "gpt-neo-1.3B-lora")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
print(result)

, Vitamins and Minerals: 1g vitamin C, 0g vitamin B12, 0g vitamin B6, 0g vitamin B2, 0g vitamin B3, 0g vitamin B5, 0g vitamin B1, 1g vitamin A, 0g vitamin D3, 0g vitamin E, 0g vitamin K, 0g iron, 0g potassium, 0g sodium, 0g sucrose, 0g zinc, 0g calcium, 0g protein.

N-3 and N-6 fatty acids are included in the nutritional needs for this diet, along with 2g of protein and 0.2g of fat. The nutritional needs for this diet are for the individual.

Nutritional and Nutritional Requirements

2g protein

0.2g fat

0g carbs

0g fat

0g protein

0.0g sugar

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.0g protein

0.0g fat

0.
